# Running the random search on randomly sampled dev set and analyzing those results

Data is only taking from the training data in these cases. 

In [1]:
%load_ext autoreload

In [126]:
%autoreload
from ast import literal_eval
import numpy as np
import pandas as pd
RESOURCE_PATH = '/data/scratch-oc40/geeticka/relation_extraction/output/%s/Final_Result/final_result_hyperparam_nocrossval.csv' # location of the output folder
def res(resource_path, dataset): return RESOURCE_PATH % dataset

In [127]:
i2b2_results = pd.read_csv(res(RESOURCE_PATH, 'i2b2'))

In [128]:
ddi_results = pd.read_csv(res(RESOURCE_PATH, 'ddi'))

In [129]:
semeval2010_results = pd.read_csv(res(RESOURCE_PATH, 'semeval2010'))

In [130]:
def get_important_stats(input_list):
    mean = round(np.mean(input_list), 2)
    median = round(np.median(input_list), 2)
    minimum = round(min(input_list), 2)
    maximum = round(max(input_list), 2)
    range_stat = round(maximum - minimum, 2)
    stddev = round(np.std(input_list), 2)
    return "Mean {}\nMedian {}\nMax {} and Min {}: Range {}\nStddev {}\n".format(mean, median, maximum, minimum, range_stat, stddev)

In [131]:
# get the 10% largest results based upon a column
def return_ten_percent_and_stats(df, column):
    def convert_to_int(row):
        col = row[column]
        try:
            col = literal_eval(col)
            return col[0]
        except ValueError:
            return float(col)
    df['assessing_col'] = df.apply(convert_to_int, axis=1)
    stats = get_important_stats(list(df['assessing_col']))
    ten_percent = int(0.1* len(df))
    new_df = df.nlargest(ten_percent, 'assessing_col').reset_index(drop=True)
    return new_df, stats

In [180]:
def print_parameters(row):
    parameters = row.Parameters
    parameters = literal_eval(parameters)['model_options']
    num_epoches = parameters['num_epoches'] # default 250
    lr_values = parameters['lr_values'] # default [0.001, 0.001, 0.001]
    lr_boundaries = parameters['lr_boundaries'] # default [60, 80]
    filter_size = parameters['filter_sizes'] # default 2,3,4,5
    batch_size = parameters['batch_size'] # default 50
    early_stop = parameters['early_stop'] # default False
    patience = parameters['patience'] # default 100
    border_size = parameters['border_size']
    
    return 'Border Size: {}\tNumber of Epoches: {}\t Learning Rate Values: {}\t Learning Rate Boundaries: {} \tFilter Size: {}\tBatch Size: {}\tEarly Stop: {}\tPatience: {}'.format(
        border_size, num_epoches, lr_values, lr_boundaries, filter_size, batch_size, early_stop, patience)
# if patience is exceeded, it is important to write this information in the dataframe. For now, I can look at 
# the ID to confirm

In [132]:
i2b2_top, i2b2_all_stats = return_ten_percent_and_stats(i2b2_results, 'Micro F1')

In [133]:
ddi_top, ddi_all_stats = return_ten_percent_and_stats(ddi_results, 'Macro F1')

In [134]:
semeval2010_top, semeval2010_all_stats = return_ten_percent_and_stats(semeval2010_results, 'Macro F1')

In [138]:
i2b2_top_stats = get_important_stats(list(i2b2_top['assessing_col']))
ddi_top_stats = get_important_stats(list(ddi_top['assessing_col']))
semeval2010_top_stats = get_important_stats(list(semeval2010_top['assessing_col']))

## Print the important statistics

### i2b2 data stats for all search vs top 10%

In [141]:
print(i2b2_all_stats)

Mean 69.61
Median 69.78
Max 72.86 and Min 62.92: Range 9.94
Stddev 1.54



In [142]:
print(i2b2_top_stats)

Mean 72.19
Median 72.13
Max 72.86 and Min 71.64: Range 1.22
Stddev 0.39



### ddi data stats for all search vs top 10%

In [143]:
print(ddi_all_stats)

Mean 80.24
Median 80.45
Max 82.57 and Min 71.21: Range 11.36
Stddev 1.63



In [144]:
print(ddi_top_stats)

Mean 82.08
Median 82.04
Max 82.57 and Min 81.74: Range 0.83
Stddev 0.25



### semeval2010 stats for all search vs top 10%

In [145]:
print(semeval2010_all_stats)

Mean 76.83
Median 79.42
Max 81.37 and Min 4.73: Range 76.64
Stddev 9.93



In [146]:
print(semeval2010_top_stats)

Mean 80.87
Median 80.74
Max 81.37 and Min 80.54: Range 0.83
Stddev 0.31



In [181]:
i2b2_top

,Micro F1,Parameters,Train Start Time,Hyperparam Tuning Mode,ID,Date of Starting Command,Execution Time (hr),assessing_col
0,"(72.86, 87.35, 64.32, 85.57, 67.77)","{'model_options': {'dataset': 'i2b2', 'preproc...",2019-03-31_19-17-43-987359,True,fbb190f6-9a50-4781-8842-8a5f205b5bf7,2019-3-31,0.632639,72.86
1,"(72.83, 87.35, 65.18, 84.16, 67.83)","{'model_options': {'dataset': 'i2b2', 'preproc...",2019-03-31_19-21-08-138195,True,c90be7f8-be71-447c-8fc6-3ac02b0aa956,2019-3-31,0.968890,72.83
2,"(72.59, 88.01, 63.68, 83.9, 69.64)","{'model_options': {'dataset': 'i2b2', 'preproc...",2019-03-31_15-55-22-95813,True,78e37b38-413b-4df5-8318-6a090a8bfe84,2019-3-31,0.620286,72.59
3,"(72.49, 86.7, 63.59, 83.17, 70.59)","{'model_options': {'dataset': 'i2b2', 'preproc...",2019-03-31_15-55-16-448413,True,1dc0d999-5244-4f36-a082-1b7c15b74b31,2019-3-31,0.532224,72.49
4,"(72.28, 87.22, 63.96, 85.0, 66.07)","{'model_options': {'dataset': 'i2b2', 'preproc...",2019-03-31_16-04-11-445318,True,2b2a8f41-f5b4-49b4-af68-37252b660a30,2019-3-31,0.764656,72.28
5,"(72.22, 86.7, 63.96, 83.17, 68.97)","{'model_options': {'dataset': 'i2b2', 'preproc...",2019-03-31_19-17-39-230741,True,8a1a5c13-5754-44eb-9f36-6c078058fe23,2019-3-31,1.485904,72.22
6,"(72.13, 87.09, 60.63, 84.88, 71.54)","{'model_options': {'dataset': 'i2b2', 'preproc...",2019-03-31_19-15-31-417749,True,ad29ddfc-64a3-4a5b-8f5e-0ff7952261b4,2019-3-31,0.735721,72.13
7,"(72.12, 87.22, 64.0, 84.42, 66.67)","{'model_options': {'dataset': 'i2b2', 'preproc...",2019-03-31_16-16-50-632594,True,a89ba918-9079-4763-b9b8-e40dbb35c76e,2019-3-31,0.294588,72.12
8,"(71.99, 87.35, 63.96, 83.74, 66.67)","{'model_options': {'dataset': 'i2b2', 'preproc...",2019-03-31_16-12-50-707712,True,83fe2674-c5f6-4aca-b80f-7004773ce1aa,2019-3-31,1.686508,71.99
9,"(71.93, 86.31, 64.84, 80.39, 70.49)","{'model_options': {'dataset': 'i2b2', 'preproc...",2019-03-31_16-00-32-372118,True,c80480cb-692f-4cde-934c-cd5926168eda,2019-3-31,0.428998,71.93


In [182]:
for index, row in i2b2_top.iterrows():
    print(print_parameters(i2b2_top.loc[index]), '\n')

Border Size: -1	Number of Epoches: 203	 Learning Rate Values: [0.001, 0.00014]	 Learning Rate Boundaries: [101] 	Filter Size: 3,4,5	Batch Size: 81	Early Stop: True	Patience: 40 

Border Size: -1	Number of Epoches: 299	 Learning Rate Values: [0.00044, 0.00044]	 Learning Rate Boundaries: [149] 	Filter Size: 2,3,4,5	Batch Size: 73	Early Stop: True	Patience: 59 

Border Size: -1	Number of Epoches: 126	 Learning Rate Values: [0.001, 0.00077]	 Learning Rate Boundaries: [63] 	Filter Size: 3,4,5,6	Batch Size: 64	Early Stop: True	Patience: 25 

Border Size: -1	Number of Epoches: 103	 Learning Rate Values: [0.001, 0.00095]	 Learning Rate Boundaries: [51] 	Filter Size: 3,4,5,6	Batch Size: 88	Early Stop: True	Patience: 20 

Border Size: -1	Number of Epoches: 246	 Learning Rate Values: [0.001, 0.00044]	 Learning Rate Boundaries: [123] 	Filter Size: 3,4,5	Batch Size: 48	Early Stop: True	Patience: 49 

Border Size: -1	Number of Epoches: 286	 Learning Rate Values: [0.001, 0.001]	 Learning Rate Boundar

In [183]:
ddi_top

,Macro F1,Parameters,Train Start Time,Hyperparam Tuning Mode,ID,Date of Starting Command,Execution Time (hr),assessing_col
0,"(82.57, 79.01, 89.2)","{'model_options': {'dataset': 'ddi', 'preproce...",2019-04-01_12-32-30-196385,True,424c2902-e700-4ad2-a350-1317f8ffa339,2019-4-1,1.559819,82.57
1,"(82.31, 78.69, 89.01)","{'model_options': {'dataset': 'ddi', 'preproce...",2019-03-31_21-26-08-476254,True,3931a469-8124-41ec-a057-66011bbb8cd9,2019-3-31,1.712886,82.31
2,"(82.28, 78.72, 88.82)","{'model_options': {'dataset': 'ddi', 'preproce...",2019-04-01_12-24-51-182232,True,515a9d3b-2bfe-4f36-a3be-e9b65d9a5b8e,2019-4-1,1.651390,82.28
3,"(82.2, 78.54, 89.15)","{'model_options': {'dataset': 'ddi', 'preproce...",2019-03-31_21-32-55-922810,True,71fdc124-b15e-443f-a4b5-2f23b646ee9b,2019-3-31,1.380437,82.20
4,"(82.1, 78.37, 89.98)","{'model_options': {'dataset': 'ddi', 'preproce...",2019-03-31_21-26-01-850589,True,3ff6efc3-7dde-4636-8304-175ff3fc9387,2019-3-31,1.084237,82.10
5,"(81.97, 78.2, 90.1)","{'model_options': {'dataset': 'ddi', 'preproce...",2019-04-01_11-21-51-831415,True,1a0785b6-87d0-482a-90b2-7510353527a0,2019-4-1,2.213494,81.97
6,"(81.96, 78.23, 89.1)","{'model_options': {'dataset': 'ddi', 'preproce...",2019-04-01_11-21-26-215335,True,f26c0b8f-b5e1-4639-969c-fac941f142ec,2019-4-1,3.510193,81.96
7,"(81.89, 78.2, 89.36)","{'model_options': {'dataset': 'ddi', 'preproce...",2019-04-01_11-23-36-838299,True,f3d4b2f4-3f9d-4400-8e4b-22f3d1cfa32c,2019-4-1,1.915579,81.89
8,"(81.78, 78.01, 89.82)","{'model_options': {'dataset': 'ddi', 'preproce...",2019-04-01_11-20-18-963137,True,f6d6def4-a1db-4e4b-bb65-0ae9eaa5ed62,2019-4-1,3.377728,81.78
9,"(81.74, 77.97, 89.47)","{'model_options': {'dataset': 'ddi', 'preproce...",2019-03-31_21-31-52-824418,True,1ecefaa8-390d-419b-abad-8756b71eee9d,2019-3-31,2.865627,81.74


In [184]:
for index, row in ddi_top.iterrows():
    print(print_parameters(ddi_top.loc[index]), '\n')

Border Size: -1	Number of Epoches: 124	 Learning Rate Values: [0.001, 0.00021]	 Learning Rate Boundaries: [62] 	Filter Size: 3,4,5	Batch Size: 37	Early Stop: True	Patience: 24 

Border Size: -1	Number of Epoches: 226	 Learning Rate Values: [0.001, 0.0003]	 Learning Rate Boundaries: [113] 	Filter Size: 3,4,5,6	Batch Size: 47	Early Stop: True	Patience: 45 

Border Size: -1	Number of Epoches: 108	 Learning Rate Values: [0.001, 9e-05]	 Learning Rate Boundaries: [54] 	Filter Size: 2,3,4,5	Batch Size: 33	Early Stop: False	Patience: 100 

Border Size: -1	Number of Epoches: 189	 Learning Rate Values: [0.00069, 0.00069]	 Learning Rate Boundaries: [94] 	Filter Size: 3,4,5,6	Batch Size: 81	Early Stop: True	Patience: 37 

Border Size: -1	Number of Epoches: 291	 Learning Rate Values: [0.001, 0.00076]	 Learning Rate Boundaries: [145] 	Filter Size: 3,4,5,6	Batch Size: 93	Early Stop: True	Patience: 58 

Border Size: -1	Number of Epoches: 171	 Learning Rate Values: [0.001, 0.00063]	 Learning Rate Bound

In [185]:
semeval2010_top

,Macro F1,Parameters,Train Start Time,Hyperparam Tuning Mode,ID,Date of Starting Command,Execution Time (hr),assessing_col
0,81.37,"{'model_options': {'dataset': 'semeval2010', '...",2019-03-31_12-22-12-980407,True,253a8196-7cb7-4898-aca5-e11c4c33f068,2019-3-31,0.595738,81.37
1,81.29,"{'model_options': {'dataset': 'semeval2010', '...",2019-03-31_12-21-54-283464,True,b0ed0a12-c1d9-498d-b81d-3d853d6bac6b,2019-3-31,0.790983,81.29
2,81.29,"{'model_options': {'dataset': 'semeval2010', '...",2019-03-31_15-47-47-925302,True,41869728-b510-4f93-a7c1-3c6391c22fae,2019-3-31,0.631535,81.29
3,80.85,"{'model_options': {'dataset': 'semeval2010', '...",2019-03-31_12-27-40-96508,True,d0636bb4-4f1e-46ee-8d13-a2bb5501b55c,2019-3-31,0.466067,80.85
4,80.80,"{'model_options': {'dataset': 'semeval2010', '...",2019-03-31_12-18-30-932974,True,6a8f4aa4-9a3b-4e01-8e2a-b4d217765448,2019-3-31,0.621840,80.80
5,80.69,"{'model_options': {'dataset': 'semeval2010', '...",2019-03-31_01-02-56-901592,True,bef65790-d598-4b8b-8d16-48072c10b735,2019-3-30,0.497475,80.69
6,80.65,"{'model_options': {'dataset': 'semeval2010', '...",2019-03-31_12-31-43-522210,True,d3d9f8bc-ed5c-4b69-8faa-71ac0d84db21,2019-3-31,0.633159,80.65
7,80.64,"{'model_options': {'dataset': 'semeval2010', '...",2019-03-31_12-14-27-634127,True,2c5d6b3a-3fd1-4c77-9ae8-4df79bc2f747,2019-3-31,0.585461,80.64
8,80.55,"{'model_options': {'dataset': 'semeval2010', '...",2019-03-31_01-03-00-285695,True,bbe5eb4d-937a-4548-b76c-fc441ec96a8e,2019-3-30,0.662352,80.55
9,80.54,"{'model_options': {'dataset': 'semeval2010', '...",2019-03-31_01-16-13-881486,True,2d69e01c-3da8-4078-9f51-1e70b66e7f10,2019-3-30,0.447052,80.54


In [186]:
for index, row in semeval2010_top.iterrows():
    print(print_parameters(semeval2010_top.loc[index]), '\n')

Border Size: 1	Number of Epoches: 283	 Learning Rate Values: [0.001, 0.0005]	 Learning Rate Boundaries: [141] 	Filter Size: 3,4,5,6	Batch Size: 88	Early Stop: False	Patience: 100 

Border Size: 1	Number of Epoches: 226	 Learning Rate Values: [0.00087, 0.00087]	 Learning Rate Boundaries: [113] 	Filter Size: 3,4,5,6	Batch Size: 35	Early Stop: False	Patience: 100 

Border Size: 50	Number of Epoches: 163	 Learning Rate Values: [0.001, 0.001]	 Learning Rate Boundaries: [81] 	Filter Size: 3,4,5	Batch Size: 73	Early Stop: True	Patience: 32 

Border Size: 1	Number of Epoches: 272	 Learning Rate Values: [0.001, 0.00015]	 Learning Rate Boundaries: [136] 	Filter Size: 3,4,5,6	Batch Size: 90	Early Stop: True	Patience: 54 

Border Size: -1	Number of Epoches: 254	 Learning Rate Values: [0.00055, 0.00055]	 Learning Rate Boundaries: [127] 	Filter Size: 2,3,4,5	Batch Size: 65	Early Stop: False	Patience: 100 

Border Size: 50	Number of Epoches: 166	 Learning Rate Values: [0.00048, 0.00048]	 Learning Rat